# Simple Vortex in 2D

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import *                # Conveniences like clear_output

Here we generate some vortices.  These are regularized by fixing the coupling constant $g$ so that the homogeneous system in the box and on the lattice gives a fixed value of $\Delta$ at the specified chemical potential.  The self-consistent solution is found by simple iterations.

In [ ]:
from mmf_hfb import bcs, homogeneous;reload(bcs)
from mmfutils.math.special import mstep
from mmfutils.plot import imcontourf

class Vortex(bcs.BCS):
    barrier_width = 0.2
    barrier_height = 100.0
    
    def __init__(self, Nxyz=(32, 32), Lxyz=(10., 10.), **kw):
        self.R = min(Lxyz)/2
        bcs.BCS.__init__(self, Nxyz=Nxyz, Lxyz=Lxyz, **kw)
    
    def get_v_ext(self):
        r = np.sqrt(sum([_x**2 for _x in self.xyz[:2]]))
        R0 = self.barrier_width * self.R
        V = self.barrier_height * mstep(r-self.R+R0, R0)
        return (V, V)

class VortexState(Vortex):
    def __init__(self, mu, dmu, delta, **kw):
        Vortex.__init__(self, **kw)
        self.mus = (mu+dmu, mu-dmu)
        self.g = self.get_g(mu=mu, delta=delta)
        x, y = self.xyz
        self.Delta = delta*(x+1j*y)
        
    def get_g(self, mu=1.0, delta=0.2):
        h = homogeneous.Homogeneous(Nxyz=self.Nxyz, Lxyz=self.Lxyz) 
        res = h.get_densities(mus_eff=(mu, mu), delta=delta)
        g = delta/res.nu.n
        return g
    
    def solve(self, tol=0.01, plot=True):
        err = 1.0
        fig = None
        with NoInterrupt() as interrupted:
            display(self.plot())
            clear_output(wait=True)

            while not interrupted and err > tol:
                res = self.get_densities(mus_eff=self.mus, delta=self.Delta)
                Delta0, self.Delta = self.Delta, self.g*res.nu  # ....
                err = abs(Delta0 - self.Delta).max()
                if display:
                    plt.clf()
                    fig = self.plot(fig=fig, res=res)
                    plt.suptitle(f"err={err}")
                    display(fig)
                    clear_output(wait=True)
        
    
    def plot(self, fig=None, res=None):
        x, y = self.xyz
        if fig is None:
            fig = plt.figure(figsize=(20, 10))
        plt.subplot(231)
        imcontourf(x, y, abs(self.Delta), aspect=1)
        plt.title(r'$|\Delta|$'); plt.colorbar()
        
        if res is not None:
            plt.subplot(232)
            
            imcontourf(x, y, (res.n_a+res.n_b).real, aspect=1)
            plt.title(r'$n_+$'); plt.colorbar()
        
            plt.subplot(233)
            imcontourf(x, y, (res.n_a-res.n_b).real, aspect=1)
            plt.title(r'$n_-$'); plt.colorbar()
            
            plt.subplot(234)
            
            j_a = res.j_a[0] + 1j*res.j_a[1]
            j_b = res.j_b[0] + 1j*res.j_b[1]
            j_p = j_a + j_b
            j_m = j_a - j_b
            utheta = np.exp(1j*np.angle(x + 1j*y))
            imcontourf(x, y, abs(j_p), aspect=1)
            plt.title(r'$j_p$'); plt.colorbar()
            plt.quiver(x.ravel(), y.ravel(), j_a.real.T, j_a.imag.T)
            
            plt.subplot(235)
            imcontourf(x, y, abs(j_m), aspect=1)
            plt.title(r'$J_-$'); plt.colorbar()
            plt.quiver(x.ravel(), y.ravel(), j_m.real.T, j_m.imag.T)
        return fig      

v = Vortex(Nxyz=(16, 16))

$$
  v_x +\I v_y = v e^{\I\phi}, \\
  \uvect{\theta} = \frac{\I x - y}{r}\\
  \uvect{\theta}\cdot \vect{v} = \frac{-yv_x + xv_y}{r} = - \Re(v \uvect{\theta})
$$

# Potential Profile

In [ ]:
V = v.get_v_ext()[0]
x , y = v.xyz
imcontourf(x, y, V, aspect=1);plt.colorbar()

# Symmetric Vortex

In [ ]:
v0 = VortexState(mu=10.0, dmu=0.0, delta=1.0, Nxyz=(32, 32))
v0.solve(plot=True)

# Polarized Vortex

In [ ]:
v1 = VortexState(mu=10.0, dmu=2, delta=5.0, Nxyz=(32, 32))
v1.solve(plot=True)

## Compare to FF State

In [ ]:
from mmf_hfb import FuldeFerrelState; reload(FuldeFerrelState)
N = v.Nxyz[0]
L = v.Lxyz[0]
k_c = np.sqrt(2)*np.pi*N/L
mu = 10.0
dmu = 2.0
delta = 5.0
args = dict(mu=mu, dmu=dmu, delta=delta, k_c=k_c)
f = FuldeFerrelState.FFState(fix_g=True, **args)
rs = np.linspace(0.01,1, 10)
rs = np.append(rs, np.linspace(1.1, 3, 15))
ds = [f.solve(mu=mu, dmu=dmu, dq=1/_r, a=0.001, b=2*delta) for _r in rs]

In [ ]:
r = np.sqrt(sum(_x**2 for _x in v1.xyz))
plt.plot(r.ravel(), abs(v1.Delta).ravel(), '+')
plt.plot(rs, ds)
plt.xlabel("r", fontsize=20)
plt.ylabel(r'$\Delta$', fontsize=20)

In [ ]:
na = np.array([])
nb = np.array([])
res = v1.get_densities(mus_eff=v1.mus, delta=v1.Delta)
for i in range(len(rs)):
    na_, nb_ = f.get_densities(delta=ds[i], dq=1/rs[i], mu=mu, dmu=dmu)
    na = np.append(na, na_.n)
    nb = np.append(nb, nb_.n)   

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(121)
n_p = na + nb
plt.plot(r.ravel(), abs(res.n_a + res.n_b).ravel(), '+')
plt.plot(rs, n_p)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$n_p$", fontsize=20)
plt.title("Total Density")

plt.subplot(122)
n_m = na - nb
plt.plot(r.ravel(), abs(res.n_a - res.n_b).ravel(), '+')
plt.plot(rs, n_m)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$n_m$", fontsize=20),plt.title("Density Difference")
clear_output()

In [ ]:
ja = np.abs(res.j_a[0] + 1j*res.j_a[1])
jb = np.abs(res.j_b[0] + 1j*res.j_b[1]) 
j_p_, j_m_ = ja + jb, ja - jb
ja = []
jb = []
for i in range(len(rs)):
    ja.append(na[i]/rs[i])
    jb.append(nb[i]/rs[i])
ja, jb = np.array(ja), np.array(jb)
j_p, j_m = ja + jb, ja - jb
plt.figure(figsize=(20, 5))
plt.subplot(121)
plt.plot(r.ravel(), 2*j_p_.ravel(), '+')
plt.plot(rs, j_p)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$j_p$", fontsize=20),plt.title("Total Current")
plt.subplot(122)
plt.plot(r.ravel(), 2*j_m_.ravel(), '+')
plt.plot(rs, j_m)
plt.xlabel("r", fontsize=20), plt.ylabel(r"$j_m$", fontsize=20),plt.title("Current Difference")
clear_output()

## FF State Phase Diagram

In [ ]:
from mmf_hfb import FuldeFerrelState; reload(FuldeFerrelState)
kF = 1
m = 1
T = 0
q = 0
eF=kF**2/2/m
mu = 0.5 * eF

delta0 = np.sqrt(2.0) * eF
dmu = 0.75 * delta0
dmus_delta0 = np.linspace(0,2,10)
delta_delta0 = np.linspace(0,2,10)
ff = FuldeFerrelState.FFState(fix_g=True, mu=mu, dmu=dmu, delta=delta0, d=2, k_c=500, m=m, T=T)

In [ ]:
dq = 0
ds = np.linspace(0.1 * delta0, 2* delta0, 10)
fs = [ff.f(mu=mu, dmu=dmu, delta=d, dq= dq) for d in ds]
plt.plot(ds,fs,'--')
plt.axhline(0)
if(fs[0] * fs[-1] > 0):
    for i in range(len(fs)):
        if fs[0] * fs[i] < 0: #two solutions
            d = ff.solve(mu=mu, dmu=dmu, dq= dq,a=ds[0],b = ds[i])
            plt.axvline(d)
            p1 = ff.get_pressure(mu=mu, dmu=dmu, dq = dq, delta=d)
            d = ff.solve(mu=mu, dmu=dmu, dq= dq,a=ds[i],b = ds[-1])
            plt.axvline(d)
            p2 = ff.get_pressure(mu=mu, dmu=dmu, dq = dq, delta=d)
            print(f"p1={p1}\tp2={p2}")
            break
else:
    d = ff.solve(mu=mu, dmu=dmu, dq= dq)
    plt.axvline(d)

clear_output()
print(d)

In [ ]:
delta0

In [ ]:
ff.f(mu=mu, dmu=dmu, delta=0.5591259549249719, dq= dq,a=0.2,b = 1.4)

In [ ]:
dqs = np.linspace(0,1.3, 10)
ps = [ff.get_pressure(mu=mu, dmu=dmu, dq = dq).n for dq in dqs]
plt.plot(dqs, ps)
clear_output()

# Quiver

In [ ]:
x, y = VortexState(mu=10.0, dmu=0.4, delta=1.0, Nxyz=(32, 32)).xyz
v = 1j*x - 2*y
plt.quiver(x.ravel(), y.ravel(), v.T.real, v.T.imag)

$$
  -u^\dagger \I\nabla u
$$

$$
E(\psi)={\bra{\psi}\frac{-\nabla^2}{2m}+\frac{g}{2}\abs{\psi^\dagger\psi}-\mu\ket{\psi}}
$$

$$
\psi = f(r)e^{i\theta}\\
\vec{\psi'} = \nabla f(r)(x+iy) + f(r)(x, iy)
$$

$$
E(\psi)= \int dr \left[\frac{\psi'^2}{2m} + \frac{g}{2}f(r)^2-\mu n(r)\right]
$$